In [ ]:
CACHE_DIR = "./elevations"

def get_hgt_name(lat, lon, hgt_dot=True):
    retval = ''

    retval += 'N{:02d}'.format(+int(floor(lat)))
    retval += 'W{:03d}'.format(-int(floor(lon)))
    if hgt_dot:
        retval += '.'
    retval += 'hgt'

    return retval

def get_elevation_batch(lat_lon_list):
    # organize requests by filename
    req_dict = {}
    for k, (lat, lon) in enumerate(lat_lon_list):
        key = int(floor(lat)), int(floor(lon))
        if key not in req_dict:
            req_dict[key] = ([], [], [])
        req_dict[key][0].append(k)
        req_dict[key][1].append(lat)
        req_dict[key][2].append(lon)

    # process the files one-by-one
    retval = np.zeros(len(lat_lon_list))
    for (lat_int, lon_int), (indices, lats, lons) in req_dict.items():
        # find the location of the file
        filename = get_hgt_name(lat_int, lon_int)
        fullfile = CACHE_DIR / filename
        # download the file if needed
        if not fullfile.is_file():
            # if not determine the URL where it is located
            zip_name = get_zip_name(lat_int, lon_int)
            url = get_url_for_zip(zip_name)
            # then download the file
            print(f'Downloading {zip_name}')
            CACHE_DIR.mkdir(exist_ok=True, parents=True)
            download(url, CACHE_DIR / zip_name)
            # finally unzip and remove the file
            unzip(CACHE_DIR / zip_name)
        # process the file
        data = read_elevation_from_file(fullfile, np.array(lats), np.array(lons))
        # write data back into the return value at the appropriate indices
        retval[indices] = data

    # return the elevation data
    return retval